In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 28 00:30:52 2019

@author: jerryyao
"""

import numpy as np

from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from graspy.embed import casc_new
from graspy.embed import LaplacianSpectralEmbed
from graspy.simulations import sbm
from graspy.plot import heatmap, pairplot
from sklearn.metrics import adjusted_rand_score as ARI
import graspy
from Casc import *
import time
def GraphCovaritesSimulater(covProb,labels,agreement):
    time_start=time.time()
    base=np.eye(max(labels)+1)
    membership=np.zeros([len(labels),covProb.shape[1]])
    n_misassign=0
    for i in range (0,len(labels)):
        assignRV=bool(np.random.binomial(1,agreement))
        if assignRV:
            membership[i,:]=base[labels[i],:]
        else:
            membership[i,:]=base[(labels[i]+1)%(max(labels)+1),:]
            n_misassign=n_misassign+1
    agreement_=1-n_misassign/(len(labels))
    probMat=np.dot(membership,covProb)
    covMat=np.zeros(probMat.shape)
    for i in range(0,covMat.shape[0]):
        for j in range(0,covMat.shape [1]):
            covMat[i,j]=np.random.binomial(1,probMat[i,j])
    time_end=time.time()
    print('tcovgen',(time_end-time_start)*1000,'ms')
    return covMat,agreement_,membership
        
def simulation(p,q,m1,m2,assortative,CCA,agreement):
    n_verts = 1500
    labels_sbm = n_verts * [0] + n_verts * [1]+n_verts * [2]
    if assortative:
        P = np.array([[p,q,q],[q,p,q],[q,q,p]])
    else:
        P = np.array([[q,p,p],[p,q,p],[p,p,q]])
    undirected_sbm = sbm(3 * [n_verts], P)
    #    heatmap(undirected_sbm, title='2-block SBM (undirected)', inner_hier_labels=labels_sbm)
    
#    nMembers=[150,150,150]
    covProbMat = np.matrix([[m1,m2,m2],[m2,m1,m2],[m2,m2,m1]])
    
    covMat,agreement_,membership = GraphCovaritesSimulater(covProbMat,labels_sbm,agreement)
    
    time_start=time.time()
    ase = graspy.embed.casc_new.CovariateAssistedSpectralEmbed(n_components=3,assortative=True,CCA=True)
    Xhat = ase.fit(undirected_sbm,covMat) 
    loss_casc_cca=(1-ARI(Xhat['cluster'],labels_sbm))
    
    time_end=time.time()
    print('tCCA',(time_end-time_start)*1000,'ms')
    
    time_start=time.time()
    ase = graspy.embed.casc_new.CovariateAssistedSpectralEmbed(n_components=3,assortative=True)
    Xhat = ase.fit(undirected_sbm,covMat)
    loss_casc_assortative=(1-ARI(Xhat['cluster'],labels_sbm))

    time_end=time.time()
    print('tasscasc',(time_end-time_start)*1000,'ms')
    
    time_start=time.time()
    ase = graspy.embed.casc_new.CovariateAssistedSpectralEmbed(n_components=3,assortative=False)
    Xhat = ase.fit(undirected_sbm,covMat)
    loss_casc=(1-ARI(Xhat['cluster'],labels_sbm))
    
    time_end=time.time()
    print('tcasc',(time_end-time_start)*1000,'ms')

    time_start=time.time()

    lse =LaplacianSpectralEmbed(form='R-DAD',regularizer=1,n_components=3)
    X1hat = lse.fit_transform(undirected_sbm)
    kmeansResults=KMeans(n_clusters=3).fit(X1hat)
    labels=kmeansResults.labels_
    loss_regulrized_lse=(1-ARI(labels,labels_sbm))
    
    time_end=time.time()
    print('tlse',(time_end-time_start)*1000,'ms')
    
    time_start=time.time()

#pairplot(Xhat['cascSvd'], title='SBM casc embeddingCC')
    lse =LaplacianSpectralEmbed(form='R-DAD',regularizer=1,n_components=3)
    X1hat = lse.fit_transform(np.dot(covMat,covMat.transpose()))
    kmeansResults=KMeans(n_clusters=3).fit(X1hat)
    labels=kmeansResults.labels_
    loss_cov_lse=(1-ARI(labels,labels_sbm))
    
    time_end=time.time()
    print('tcov',(time_end-time_start)*1000,'ms')
#    ase = graspy.embed.casc_new.CovariateAssistedSpectralEmbed(n_components=4,assortative=True,CCA=True)
#    Xhat = ase.fit(undirected_sbm,covMat) 
#    loss_casc_cca=(1-ARI(Xhat['cluster'],labels_sbm))
    
    return[loss_cov_lse,loss_regulrized_lse,loss_casc_assortative,loss_casc,loss_casc_cca],agreement_
    

p=0.0225
q=0.015
m1=0.8
m2=0.2
loss5=np.zeros((4,5))
agreementMat=np.zeros((4,1))
for i in range(0,4):
    print(i)
    loss5[i,:],agreementMat[i]=simulation(p,q,m1,m2,True,False,0.6+i*0.1)

#    

p=0.0225
q=0.015
m1=0.8
m2=0.2
loss6=np.zeros((4,5))
agreementMat=np.zeros((4,1))
for i in range(0,4):
    print(i)
    loss6[i,:],agreementMat[i]=simulation(p,q,m1,m2,False,False,0.6+i*0.1)
   






    
p=0.015
q=0.015
m1=0.8
m2=0.2
loss=np.zeros((5,5))
for i in range(0,5):
    print(i)
    loss[i,:]=simulation(p+i*0.005,q,m1,m2,True,False,1)[0]
    
p=0.015
q=0.015
m1=0.8
m2=0.2    
loss1=np.zeros((5,5))
for i in range(0,5):
    print(i)
    loss1[i,:]=simulation(p+i*0.005,q,m1,m2,False,False,1)[0]
    
p=0.0225
q=0.015
m1=0.2
m2=0.2

loss2=np.zeros((7,5))
for i in range(0,7):
    loss2[i,:]=simulation(p,q,m1+0.1*i,m2,True,False,1)[0]
    
    
p=0.0225
q=0.015
m1=0.2
m2=0.2
    
loss3=np.zeros((7,5))
for i in range(0,7):
    loss3[i,:]=simulation(p,q,m1+0.1*i,m2,False,False,1)[0]
    
    
#    
#    
#    
# -*- coding: utf-8 -*-
"""
Created on Mon Nov  4 10:56:43 2019

@author: jerryyao
"""

import matplotlib.pyplot as plt
import numpy as np
x1=np.arange(0,0.025,0.005)
pltlabels=['cov','lse','cascassort','cascnonassort','cca']
linestyle_str = [
     ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or '.'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),
     ('longdash',(0,(10,2)))]  # Same as '-.'
linestyletuple=['r--','k-.','k--','k-','k:']

ax1=plt.subplot(3, 2, 1)
for i in range(0,5):
    ax1.plot(x1,loss[:,i],linestyletuple[i])
ax1.legend(pltlabels)
ax2=plt.subplot(3, 2, 2)
#plt.figure()
for i in range(0,5):
    ax2.plot(x1,loss1[:,i],linestyletuple[i])
ax2.legend(pltlabels)

x2=np.arange(0,0.7,0.1)

#plt.figure()
ax3=plt.subplot(3, 2, 3)
for i in range(0,5):
    ax3.plot(x2,loss2[:,i],linestyletuple[i])
ax3.legend(pltlabels)

#plt.figure()
ax4=plt.subplot(3, 2, 4)
for i in range(0,5):
    ax4.plot(x2,loss3[:,i],linestyletuple[i])
ax4.legend(pltlabels)

x3=np.arange(0.6,1.0,0.1)
#plt.figure()
ax5=plt.subplot(3, 2, 5)
for i in range(0,5):
    ax5.plot(x3,loss5[:,i],linestyletuple[i])
ax5.legend(pltlabels)
#plt.figure()
ax6=plt.subplot(3, 2, 6)
for i in range(0,5):
    ax6.plot(x3,loss6[:,i],linestyletuple[i])
ax6.legend(pltlabels)

plt.tight_layout()
plt.show()